In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess and augment the data
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(x_train)

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, Flatten, Dense

# Define the Inception Module
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    conv3x3_reduce = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)

    conv5x5_reduce = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)

    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool_proj = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(maxpool)

    inception_output = concatenate([conv1x1, conv3x3, conv5x5, maxpool_proj], axis=-1)
    return inception_output

# Define the GoogLeNet model
def googlenet(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Initial Convolution and MaxPooling
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception Modules
    x = inception_module(x, [64, 64, 128, 32, 32, 32])
    x = inception_module(x, [128, 128, 192, 96, 96, 64])

    # Add more inception modules as needed

    # Auxiliary Classifier 1
    aux1 = AveragePooling2D((5, 5), strides=(3, 3))(x)
    aux1 = Conv2D(128, (1, 1), padding='same', activation='relu')(aux1)
    aux1 = Flatten()(aux1)
    aux1 = Dense(1024, activation='relu')(aux1)
    aux1 = Dense(num_classes, activation='softmax')(aux1)

    # Inception Modules and other layers

    # Auxiliary Classifier 2
    aux2 = AveragePooling2D((5, 5), strides=(3, 3))(x)
    aux2 = Conv2D(128, (1, 1), padding='same', activation='relu')(aux2)
    aux2 = Flatten()(aux2)
    aux2 = Dense(1024, activation='relu')(aux2)
    aux2 = Dense(num_classes, activation='softmax')(aux2)

    # Main Classifier
    x = AveragePooling2D((7, 7), strides=(1, 1))(x)
    x = Flatten()(x)
    x = Dense(1000, activation='relu')(x)  # Adjust the number of neurons for your specific problem
    x = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=[x, aux1, aux2])

    return model

# Create the GoogLeNet model
input_shape = (224, 224, 3)  # Adjust the input shape according to your dataset
num_classes = 1000  # Adjust the number of classes for your specific problem
model = googlenet(input_shape, num_classes)

# Print a summary of the model
model. Summary()


In [ ]:
#Auxilary Classifiers
#Before bringing the exploration of the GoogLeNet architecture to a close, there’s one more component that was implemented by the creators of the network
#to regularise and prevent overfitting. This additional component is known as an Auxilary Classifier.

#One main problem of an extensive network is that they suffer from vanishing gradient descent. Vanishing gradient descent occurs when the update to the
#weights that arises from backpropagation is negligible within the bottom layers as a result of relatively small gradient value. Simply kept, the network stops
#learning during training.

#Auxilary Classifiers are added to the intermediate layers of the architecture, namely the third(Inception 4[a]) and sixth (Inception4[d]).

#Auxilary Classifiers are only utilised during training and removed during inference. The purpose of an auxiliary classifier is to perform a
#classification based on the inputs within the network's midsection and add the loss calculated during the training back to the total loss of the network.The loss of
#auxiliary classifiers is weighted, meaning the calculated loss is multiplied by 0.3.

In [ ]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    steps_per_epoch=len(x_train) // 64,
    epochs=100,
    validation_data=(x_test, y_test),
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f'Test Accuracy: {test_accuracy}')